In [1]:
## Importing libraries

In [2]:
import sys
from nanohub.uidl.teleport import JupyterCache, TeleportProject
from nanohub.uidl.rappture import RapptureBuilder
from nanohub.uidl.material import MaterialBuilder
from nanohub.uidl.material import MaterialContent
from nanohub.uidl.plotly import PlotlyBuilder
from nanohub.uidl.teleport import TeleportElement, TeleportContent
from nanohub.uidl.app import AppBuilder, FormHelper
from nanohub.uidl.nanohub import Auth

TOOLNAME="crystal_viewer"
STATE_LOADER_STATUS = "loader_status"
STATE_LOADER_OPEN = "loader_open"
STATE_ERROR_STATUS = "error_status"
STATE_ERROR_OPEN = "error_open"


In [3]:
## Creating the project and adding GUI variables 

In [4]:
Project = TeleportProject("Crystal Viewer App")
Component = Project.root
Component.addStateVariable("DrawerIsVisible", {"type":"boolean", "defaultValue": True})
#Component.addStateVariable("AppBarStyle", {"type":"string", "defaultValue": "bar_shifted"})
Component.addStateVariable("open_results", {"type":"boolean", "defaultValue": False})
Component.addStateVariable("open_params", {"type":"boolean", "defaultValue": True})
Component.addStateVariable("open_planes", {"type":"boolean", "defaultValue": False})


In [5]:
## Getting session information

In [6]:
import os
auth_data = {
    'grant_type' : 'tool',
}
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"]

In [7]:
## Creating Error

In [8]:
ErrorMessage = RapptureBuilder.Error( 
    Component, 
    error_status = STATE_ERROR_STATUS,
    error_open = STATE_ERROR_OPEN,
)

In [9]:
## Creating Loader

In [10]:
Loader = RapptureBuilder.Loader( 
    Component, 
    loader_status = STATE_LOADER_STATUS,
    loader_open = STATE_LOADER_OPEN,
)

In [11]:
## Schema

In [12]:
RapptureBuilder.buildSchema(
    Project,
    Component,
    url = "https://nanohub.org/api/tools",
    toolname = TOOLNAME
)
Component.addPropVariable("onLoadSchema", {"type":"func", 'defaultValue' : '(e)=>{e.setState({"'+ STATE_LOADER_OPEN +'":false})}'})    

In [13]:
## Authentication

In [14]:
Login, CLogin = Auth.Session(
    Project,
    Component,
    sessiontoken = auth_data["sessiontoken"],
    sessionnum = auth_data["sessionnum"],
    url = "https://nanohub.org/api/developer/oauth/token",   
)
Login.content.events["onError"]=[
    { "type": "stateChange", "modifies": STATE_ERROR_OPEN, "newState": True},
    { "type": "stateChange", "modifies": STATE_ERROR_STATUS, "newState": '$e'},
]


Login.content.events["onAuth"] = [ 
    { "type": "propCall2", "calls": "buildSchema", "args": ['self'] },
]


In [15]:
Drawer = MaterialBuilder.Drawer(Project, state="DrawerIsVisible", variant="persistent" )

In [16]:
def DiamondIcon():
    return "M 1 6 l 11 17 l 10 -17 l -3 -5 l -14 0 l -3 5 l 20 0 M 6 1 l 3 5 l 3 -5 l 3 5 l 3 -5 M 8 6 l 4 17 l 4 -17"

def ZincblendeIcon():
    return "M 0 8 l 2 10 l 7 6 l 10 -5 l 3 -3 l 0 -12 l -11 -3 l -6 1 l -5 7 M 1 9 l 8 -4 l 3 -4 M 1 9 l 7 7 l 8 -4 l 6 -8 M 8 15 l 2 9 M 9 5 l 8 7 l 1 7"

def WurtziteIcon():
    return "M 1 8 l 7 2 l 9 -1 l 5 -5 l -7 -2 l -10 1 l -4 5 M 1 22 l 7 2 l 9 -1 l 5 -5 M 1 8 v 14 M 8 10 v 14 M 17 9 v 14 M 22 4 v 14"

def RockIcon():
    return "M 1 5 l 8 6 l 14 -3 l -8 -6 l -14 3 M 1 18 l 8 6 l 14 -3 M 1 5  v 13 M 9 11 v 13 M 23 8 v 13M 12 4.5 a 6 2 0 1 0 0.01 0M 3 10 a 5 2 60 1 0 0.01 0 M 17 12 a 4 5 60 1 0 0.01 0"

def CesiumIcon():
    return "M 1 3 l 10 2 l 10 -2 l -10 -2 l -10 2 M 1 17 l 10 5 l 10 -5 M 1 3 v 14 M 21 3 v 14 M 11 5 v 17 M 11 6 a 5 5 0 1 0 0.01 0"

def FaceIcon():
    return "M 1 7 l 15 0 l 5 -5 l -15 0 l -5 5 v 15 M 1 22 l 15 0 l 5 -5 v -15 M 16 7 v 15"

def BodyIcon():
    return "M 1 7 l 15 0 l 5 -5 l -15 0 l -5 5 v 15 M 1 22 l 15 0 l 5 -5 v -15 M 16 7 v 15 M 10 12 a 1 1 0 1 0 0.01 0"

#def CarbonIcon():
#    return "M 2 5 l 0 4 l 4 3 l 4 -3 l 0 -4 l -4 -3 l -4 3 M 10 9  l 4 3 l 4 -3 l 0 -4 l -4 -3 l -4 3 M 6 12 l 0 4 l 4 3 l 4 -3 M 14 12 l 0 4 l 4 3 l 4 -3 l 0 -4 l -4 -3 M 6 1 a 1 1 0 1 0 0.01 0 M 14 1 a 1 1 0 1 0 0.01 0 M 2 4 a 1 1 0 1 0 0.01 0 M 10 4 a 1 1 0 1 0 0.01 0 M 18 4 a 1 1 0 1 0 0.01 0 M 2 8 a 1 1 0 1 0 0.01 0 M 10 8 a 1 1 0 1 0 0.01 0 M 18 8 a 1 1 0 1 0 0.01 0 M 6 11 a 1 1 0 1 0 0.01 0 M 14 11 a 1 1 0 1 0 0.01 0 M 22 11 a 1 1 0 1 0 0.01 0 M 6 15 a 1 1 0 1 0 0.01 0 M 14 15 a 1 1 0 1 0 0.01 0 M 22 15 a 1 1 0 1 0 0.01 0 M 10 18 a 1 1 0 1 0 0.01 0 M 18 18 a 1 1 0 1 0 0.01 0"

def RhomboIcon():
    return "M 1 4 l 4 3 l 11 0 l 7 -3 l -4 -3 l -12 0 l -6 3 v 16 M 1 20 l 4 3 l 11 0 l 7 -3 v -16 M 5 7 v 16 M 16 7 v 16"

#def PerovskiteIcon():
#    return "M 1 11 l 11 12 l 11 -11 l -11 -11 l -11 10 l 7 2 l 15 -1 M 12 1 l -4 12 l 4 10"

def TMDIcon():
    return "M 2 5 l 0 4 l 4 3 l 4 -3 l 0 -4 l -4 -3 l -4 3 M 10 9  l 4 3 l 4 -3 l 0 -4 l -4 -3 l -4 3 M 6 12 l 0 4 l 4 3 l 4 -3 M 14 12 l 0 4 l 4 3 l 4 -3 l 0 -4 l -4 -3 M 6 1 a 1 1 0 1 0 0.01 0 M 14 1 a 1 1 0 1 0 0.01 0 M 2 4 a 1 1 0 1 0 0.01 0 M 10 4 a 1 1 0 1 0 0.01 0 M 18 4 a 1 1 0 1 0 0.01 0 M 2 8 a 1 1 0 1 0 0.01 0 M 10 8 a 1 1 0 1 0 0.01 0 M 18 8 a 1 1 0 1 0 0.01 0 M 6 11 a 1 1 0 1 0 0.01 0 M 14 11 a 1 1 0 1 0 0.01 0 M 22 11 a 1 1 0 1 0 0.01 0 M 6 15 a 1 1 0 1 0 0.01 0 M 14 15 a 1 1 0 1 0 0.01 0 M 22 15 a 1 1 0 1 0 0.01 0 M 10 18 a 1 1 0 1 0 0.01 0 M 18 18 a 1 1 0 1 0 0.01 0 M 10 17 a 2 2 0 1 0 0.01 0 M 18 17 a 2 2 0 1 0 0.01 0 M 2 3 a 2 2 0 1 0 0.01 0 M 10 3 a 2 2 0 1 0 0.01 0 M 18 3 a 2 2 0 1 0 0.01 0 M 6 10 a 2 2 0 1 0 0.01 0 M 14 10 a 2 2 0 1 0 0.01 0 M 22 10 a 2 2 0 1 0 0.01 0"

def BlackIcon():
    return "M 1 10 l 5 3 l 4 11 l 8 -1 l -4 -3 l 8 -1 l -5 -3 l -4 -9 l -8 1 l 4 1 l -8 1 M 9 9 l 5 11 M 13 7 l -4 -1 M 2 9 a 1 1 0 1 0 0.01 0 M 6 12 a 1 1 0 1 0 0.01 0 M 10 23 a 1 1 0 1 0 0.01 0 M 18 22 a 1 1 0 1 0 0.01 0 M 14 19 a 1 1 0 1 0 0.01 0 M 22 18 a 1 1 0 1 0 0.01 0 M 17 15 a 1 1 0 1 0 0.01 0 M 13 6 a 1 1 0 1 0 0.01 0 M 6 7 a 1 1 0 1 0 0.01 0 M 8 8 a 1 1 0 1 0 0.01 0 M 10 5 a 1 1 0 1 0 0.01 0 M 9 16 a 1 1 0 1 0 0.01 0 M 9 15 a 2 2 0 1 0 0.01 0 M 13 13 a 1 1 0 1 0 0.01 0 M 13 12 a 1.5 1.5 0 1 0 0.01 0"

def CubicIcon():
    return  '''M 3 8 l 11 0 l 5 -5 l -11 0 l -5 5 v 11 M 3 19 l 11 0 l 5 -5 v -11 M 14 8 v 11 M 19 14 h -11 l -5 5 M 8 16 v -11 
            M 3 7 a 1 1 0 1 0 0.01 0 
            M 14 7 a 1 1 0 1 0 0.01 0 
            M 19 2 a 1 1 0 1 0 0.01 0 
            M 8 2 a 1 1 0 1 0 0.01 0 
            M 3 18 a 1 1 0 1 0 0.01 0 
            M 14 18 a 1 1 0 1 0 0.01 0 
            M 19 13 a 1 1 0 1 0 0.01 0 
            M 8 13 a 1 1 0 1 0 0.01 0'''.replace('\n', ' ').replace('\r', '')

def TriclinicIcon():
    return '''M 6 8 l 7 0 l 6 -5 l -7 0 l -6 5 l -3 11 l 7 0 l 6 -5 l -7 0 l -6 5 M 13 8 l -3 11 M 19 3 l -3 11 M 12 3 l -3 11
            M 6 7 a 1 1 0 1 0 0.01 0
            M 13 7 a 1 1 0 1 0 0.01 0
            M 19 2 a 1 1 0 1 0 0.01 0
            M 12 2 a 1 1 0 1 0 0.01 0
            M 3 18 a 1 1 0 1 0 0.01 0
            M 10 18 a 1 1 0 1 0 0.01 0
            M 16 13 a 1 1 0 1 0 0.01 0
            M 9 13 a 1 1 0 1 0 0.01 0'''.replace('\n', ' ').replace('\r', '')

def MonoclinicIcon():
    return '''M 3 8 h 7 l 9 -5 h -7 l -9 5 v 11 h 7 l 9 -5 h -7 l -10 5 M 19 3 v 11 M 12 3 v 11 M 10 8 v 11 
            M 3 7 a 1 1 0 1 0 0.01 0 
            M 10 7 a 1 1 0 1 0 0.01 0 
            M 19 2 a 1 1 0 1 0 0.01 0 
            M 12 2 a 1 1 0 1 0 0.01 0 
            M 3 18 a 1 1 0 1 0 0.01 0 
            M 10 18 a 1 1 0 1 0 0.01 0 
            M 19 13 a 1 1 0 1 0 0.01 0 
            M 12 13 a 1 1 0 1 0 0.01 0'''.replace('\n', ' ').replace('\r', '')

def OrthorhombicIcon():
    return '''M 6 8 h 9 l 5 -5 h -9 l -5 5 v 11 h 9 l 5 -5 h -9 l -5 5 M 15 8 v 11 M 20 3 v 11 M 11 3 v 11  
            M 6 7 a 1 1 0 1 0 0.01 0
            M 15 7 a 1 1 0 1 0 0.01 0     
            M 20 2 a 1 1 0 1 0 0.01 0     
            M 11 2 a 1 1 0 1 0 0.01 0
            M 6 18 a 1 1 0 1 0 0.01 0
            M 15 18 a 1 1 0 1 0 0.01 0     
            M 20 13 a 1 1 0 1 0 0.01 0     
            M 11 13 a 1 1 0 1 0 0.01 0'''.replace('\n', ' ').replace('\r', '')

def TetragonalIcon():
    return '''M 3 10 h 12 l 4 -5 h -12 l -4 5 v 8 h 12 l 4 -5 h -12 l -4 5 M 15 10 v 8 M 19 5 v 8 M 7 5 v 8
            M 3 9 a 1 1 0 1 0 0.01 0  
            M 15 9 a 1 1 0 1 0 0.01 0  
            M 19 4 a 1 1 0 1 0 0.01 0      
            M 7 4 a 1 1 0 1 0 0.01 0        
            M 3 17 a 1 1 0 1 0 0.01 0  
            M 15 17 a 1 1 0 1 0 0.01 0  
            M 19 12 a 1 1 0 1 0 0.01 0      
            M 7 12 a 1 1 0 1 0 0.01 0'''.replace('\n', ' ').replace('\r', '')

def HexagonalIcon():
    return '''M 5 6 l 5 3 h 6 l 5 -3 l -5 -3 h -6 l -5 3 v 10 l 5 3 h 6 l 5 -3 l -5 -3 h -6 l -5 3 M 10 9 v 10 l 6 -6 v -10  l -6 6  M 16 9 v 10 l -6 -6 v -10 l 6 6 M 5 16 h 16 v -10 h -16 M 14 6 v 10
            M 5 5 a 1 1 0 1 0 0.01 0      
            M 10 8 a 1 1 0 1 0 0.01 0   
            M 16 8 a 1 1 0 1 0 0.01 0      
            M 21 5 a 1 1 0 1 0 0.01 0 
            M 16 2 a 1 1 0 1 0 0.01 0  
            M 10 2 a 1 1 0 1 0 0.01 0           
            M 5 14 a 1 1 0 1 0 0.01 0      
            M 10 18 a 1 1 0 1 0 0.01 0   
            M 16 18 a 1 1 0 1 0 0.01 0      
            M 21 15 a 1 1 0 1 0 0.01 0 
            M 16 12 a 1 1 0 1 0 0.01 0  
            M 10 12 a 1 1 0 1 0 0.01 0'''.replace('\n', ' ').replace('\r', '')

def TrigonalIcon():
    return '''M 2 17 h 7 l 5 -7 h -7 l -5 7 l 5 -3
            M 7 14 h 7 l 5 -7 h -7 l -5 7 
            M 7 10 l 5 -3
            M 9 17 l 5 -3
            M 14 10 l 5 -3
            M 3 16 a 1 1 0 1 0 0.01 0          
            M 9 16 a 1 1 0 1 0 0.01 0  
            M 14 9 a 1 1 0 1 0 0.01 0  
            M 7 9 a 1 1 0 1 0 0.01 0
            M 8 13 a 1 1 0 1 0 0.01 0          
            M 14 13 a 1 1 0 1 0 0.01 0  
            M 19 6 a 1 1 0 1 0 0.01 0  
            M 12 6 a 1 1 0 1 0 0.01 0  '''.replace('\n', ' ').replace('\r', '')



In [17]:
SETTINGS = {
    'What_to_do': {
        'type': 'IntSwitch',
        'default_value': '1',
        'options': {
            '1': 'Material Viewer',
            '3': 'Bravais Viewer'
        },
        'units': None,
        'label': '',
        'description': ' '
    },    
    'Crystal_structure': { 
        'type': 'IconList','default_value': '1', 'units': None, 'label': 'Crystal_structure',  
        'description': 'Choose an available crystal structure.', 
        'options': {
          '1': { 'name':'Diamond', "icon": DiamondIcon()},
          '2': { 'name':'Zincblende', "icon": ZincblendeIcon()},
          '3': { 'name':'Wurtzite', "icon": WurtziteIcon()},
          '4': { 'name':'Rock Salt',"icon": RockIcon()},
          '5': { 'name':'Cesium Chloride',"icon": CesiumIcon()},
          '6': { 'name':'Face-centered',"icon": FaceIcon()},
          '7': { 'name':'Body-centered',"icon": BodyIcon()},
          #'8': { 'name':'Carbon meshes',"icon": CarbonIcon()},
          '9': { 'name':'Rhombohedral',"icon": RhomboIcon()},
          #'10': { 'name':'Perovskite',"icon": PerovskiteIcon()},
          '11': { 'name':'TMD',"icon": TMDIcon()},
          '12': { 'name':'Black Phosphorous',"icon": BlackIcon()},
        },
    },
    '1': { 'type': 'ButtonList','default_value': 'Si','units': None,'label': 'Material','description': 'Select the material', 'options': {
      'Si': { 'name':'Silicon' },
      'Ge': { 'name': 'Germanium' }
    }},
    '2': {'type': 'ButtonList', 'default_value': 'GaAs', 'units': None,'label': 'Material', 'description': 'Select the material', 'options': {
      'AlP': { 'name':'Aluminium phosphide' },
      'AlAs': { 'name':'Aluminium arsenide' },
      'AlSb': { 'name':'Aluminium antimonide' },
      'GaAs': { 'name':'Gallium arsenide' },
      'GaP': { 'name':'Gallium phosphide' },
      'GaSb': { 'name':'Gallium antimonide' },
      'InAs': { 'name':'Indium arsenide' },
      'InP': { 'name':'Indium phosphide' },
      'InSb': { 'name':'Indium antimonide' }
    }},
    '3': {'type': 'ButtonList', 'default_value': 'GaN', 'units': None, 'label': 'Material', 'description': 'Select the material', 'options': {
      'AlN': { 'name':'Aluminium nitride' },
      'InN': { 'name':'Indium nitride' },
      'GaN': { 'name':'Gallium nitride' },
    }},
    '4': {'type': 'ButtonList', 'default_value': 'NaCl', 'units': None, 'label': 'Material', 'description': 'Select the material', 'options': {
      'NaCl': { 'name':'Sodium chloride' },
      'SmSe': { 'name':'Samarium selenide' },
    }},
    '5': {'type': 'ButtonList', 'default_value': 'CsCl', 'units': None,'label': 'Material','description': 'Select the material', 'options': {
        'CsCl': { 'name':'Caesium chloride' }
    }},
    '6': {'type': 'ButtonList', 'default_value': 'Cu','units': None,'label': 'Material','description': 'Select the material','options': {
      'Cu': { 'name':'Copper ' },
      'Al': { 'name':'Aluminium' },
      'Ag': { 'name':'Silver' },
      'Au': { 'name':'Gold' },
    }},
    '7': {'type': 'ButtonList', 'default_value': 'W','units': None,'label': 'Material','description': 'Select the material', 'options': {
        'W': { 'name':'Tungsten' },
    }},
    '8': {'type': 'ButtonList', 'default_value': '1','units': None,'label': 'Material','description': 'Select the material', 'options': {
      '1': { 'name':'Graphene', },
      '2': { 'name':'Carbon nanotube', },
      '3': { 'name':'Bucky ball (C60)' },
    }},
    '9': {'type': 'ButtonList','default_value': 'Bi2Te3','units': None,'label': 'Material','description': 'Select the material','options': {
      'Bi2Te3': { 'name':'Bismuth telluride' },
    }},
    '10': {'type': 'ButtonList','default_value': 'SrTiO3','units': None,'label': 'Material','description': '','options': {
      'SrTiO3': { 'name':'Strontium titanate' },
    }},
    '11': {'type': 'ButtonList','default_value': 'MoS2','units': None,'label': 'Material','description': '','options': {
      'MoS2': { 'name':'Molybdenum disulfide' },
    }},
    '12': {'type': 'ButtonList','default_value': 'Black Phosphorous','units': None,'label': 'Material','description': '','options': {
      'Black Phosphorous': { 'name':'Black Phosphorous' },
    }},
    'Crystal_system': {
        'type': 'IconList',
        'default_value': '25',
        'options': {
          '21':  { 'name':'Triclinic', 'icon': TriclinicIcon()},
          '22': { 'name':'Monoclinic', 'icon': MonoclinicIcon()},
          '23': { 'name':'Orthorhombic', 'icon': OrthorhombicIcon()},
          '24': { 'name':'Tetragonal', 'icon': TetragonalIcon()},
          '25': { 'name':'Cubic', 'icon': CubicIcon()},
          '26': { 'name':'Hexagonal', 'icon': HexagonalIcon()},
          '27': { 'name':'Trigonal', 'icon': TrigonalIcon()},
        },
        'units': None,
        'label': 'Choose a crystal system',
        'description': 'choose one of the seven crystal systems'
    },
    '21': {
        'type': 'Select',
        'default_value': 'triclinic',
        'units': None,
        'label': 'Bravais lattice',
        'description': 'choose a bravais lattice',
        'options': {
          'triclinic': 'Simple triclinc'
        },
    },
    '22': {
        'type': 'Select',
        'default_value': 'monoclinic',
        'options': {
          'monoclinic': 'Simple monoclinic',
          'monoclinicC': 'Base-centered monoclinic'
        },
        'units': None,
        'label': 'Bravais lattice',
        'description': 'choose a bravais lattice'
    },
    '23': {
        'type': 'Select',
        'default_value': 'orthorhombic',
        'options': {
          'orthorhombic': 'Simple Orthorhombic',
          'orthorhombicC': 'Base-centered Orthorhombic',
          'orthorhombicI': 'Body-centered Orthorhombic',
          'orthorhombicF': 'Face-centered Orthorhombic'
        },
        'units': None,
        'label': 'Bravais lattice',
        'description': 'choose a bravais lattice'
    },
    '24': {
        'type': 'Select',
        'default_value': 'tetragonal',
        'options': {
          'tetragonal': 'Simple tetragonal',
          'tetragonalI': 'Body-centered tetragonal'
        },
        'units': None,
        'label': 'Bravais lattice',
        'description': 'choose a bravais lattice'
    },
    '25': {
        'type': 'Select',
        'default_value': 'simplecubic',
        'options': {
          'simplecubic': 'Simple cubic',
          'bcc': 'Body-centered cubic',
          'fcc': 'Face-centered cubic'
        },
        'units': None,
        'label': 'Bravais lattice',
        'description': 'choose a bravais lattice'
    },
        '26': {
        'type': 'Select',
        'default_value': 'hexagonal',
        'options': {
          'hexagonal': 'Simple hexagonal'
        },
        'units': None,
        'label': 'Bravais lattice',
        'description': 'choose a bravais lattice'
    },
        '27': {
        'type': 'Select',
        'default_value': 'rhombohedral',
        'options': {
          'rhombohedral': 'Simple trigonal'
        },
        'units': None,
        'label': 'Bravais lattice',
        'description': 'choose a bravais lattice'
    },
    'Nx': {
        'type': 'Integer',
        'default_value': '3',
        'units': None,
        'min': 1,
        'max': None,
        'label': 'm',
        'description': ''
    },
    'Ny': {
        'type': 'Integer',
        'default_value': '3',
        'units': None,
        'min': 1,
        'max': None,
        'label': 'n',
        'description': ''
    },
    'Nz': {
        'type': 'Integer',
        'default_value': '3',
        'units': None,
        'min': 1,
        'max': None,
        'label': 'p',
        'description': ''
    },    
    'a': {
        'type': 'NumberAsString',
        'default_value': '0.45nm',
        'units' : 'nm',
        'label': 'a',
        'description': '',
        'min': 0.1,
        'max': 10,        
    },
    'b': {
        'type': 'NumberAsString',
        'default_value': '0.35nm',
        'units' : 'nm',
        'label': 'b',
        'description': '',
        'min': 0.1,
        'max': 10,
    },
    'c': {
        'type': 'NumberAsString',
        'default_value': '0.4nm',
        'label': 'c',
        'units' : 'nm',
        'description': '',
        'min': 0.1,
        'max': 10,
    },
    'Alpha': {
        'type': 'IntegerAsString',
        'default_value': '75deg',
        'label': 'Alpha',
        'units' : 'deg',
        'description': '',
        'min': -90,
        'max': 90,  
    },
    'Beta': {
        'type': 'IntegerAsString',
        'default_value': '35deg',
        'label': 'Beta',
        'units' : 'deg',
        'description': '',
        'min': -90,
        'max': 90,  
    },
    'Gamma': {
        'type': 'IntegerAsString',
        'default_value': '65deg',
        'label': 'Gamma',
        'units' : 'deg',
        'description': '',
        'min': -90,
        'max': 90,  
    },
    'Primitive_cell': {
        'type': 'String',
        'default_value': "no",
        'description': 'Toggle on to use the primitive Bravais vector or remain it off to use the conventinal Bravais vector; for some materials, only the primitive Bravais vector is available',
        'label': 'Use primitive Bravais vector'
    },
    'Lx': {
        'type': 'Integer',
        'default_value': '3',
        'units': None,
        'min': 1,
        'max': 15,
        'label': 'Lx',
        'description': 'extension in X direction in terms of a0'
    },
    'Ly': {
        'type': 'Integer',
        'default_value': '3',
        'units': None,
        'min': 1,
        'max': 15,
        'label': 'Ly',
        'description': 'extension in Y direction in terms of a1'
    },
        'C_C_bong_length': {
        'type': 'String',
        'default_value': '0.142nm',
        'label': 'C-C bond length (A,bohr,nm)',
        'description': 'enter a number between 1e-10 nm to 1000nm with units of length (A, bohr, in, m)'
    },
    'Number_of_sheet': {
        'type': 'Integer',
        'default_value': '3',
        'units': None,
        'min': 1,
        'max': 15,
        'label': 'Number of sheet',
        'description': 'set the number of graphene sheet;please enter a integer between 1 and 15'
    },
    'Separation_distance': {
        'type': 'String',
        'default_value': '0.335nm',
        'label': 'Layer separation distance (A,bohr,nm)',
        'description': 'set the speration distance between two graphene layer;enter a number between 1e-10 nm to 1000nm with units of length (A, bohr, in, m)'
    },
    'AA_Stacking': {
        'type': 'Boolean',
        'default_value': False,
        'description': 'default is AB stacking;select to be true for AA stacking',
        'label': 'AA Stacking'
    },
    'n': {
        'type': 'Integer',
        'default_value': '3',
        'units': None,
        'min': 1,
        'max': 15,
        'label': 'n',
        'description': 'the chiral vector can be written as R = na1+ma2;the two coefficients n and m, decide the dimension of the carbon nano tube'
    },
    'm': {
        'type': 'Integer',
        'default_value': '2',
        'units': None,
        'min': 0,
        'max': 15,
        'label': 'm (<=n)',
        'description': 'the chiral vector can be written as R = na1+ma2;the two coefficients n and m, decide the dimension of the carbon nano tube'
    },
    'num_of_unit_cell': {
        'type': 'Integer',
        'default_value': '3',
        'units': None,
        'min': 1,
        'max': 15,
        'label': 'Number of unit celld',
        'description': 'number of unit cells along the direction of the tube'
    },
} 

In [18]:
## Defining Layout

In [19]:
LAYOUT = {
  'input': {
    'type': 'group',
    'id': '',
    'label': '',
    'layout': '',
    'children': [
      { 'type': 'choice', 'id': 'What_to_do', 'label': 'What_to_do', 'enable' : None},
      { 'type': 'choice', 'id': 'Crystal_structure', 'label': 'Choose a crystal structure', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
      ]}, 
      { 'type': 'choice', 'id': '1', 'label': 'Material','enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '1'},
      ]}, 
      { 'type': 'choice', 'id': '2', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '2'},
      ]}, 
      { 'type': 'choice', 'id': '3', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '3'},
      ]}, 
      { 'type': 'choice', 'id': '4', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '4'},
      ]}, 
      { 'type': 'choice', 'id': '5', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '5'},
      ]}, 
      { 'type': 'choice', 'id': '6', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '6'},
      ]}, 
      { 'type': 'choice', 'id': '7', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '7'},
      ]}, 
      { 'type': 'choice', 'id': '8', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '8'},
      ]},
      { 'type': 'choice', 'id': '9', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '9'},
      ]},
      { 'type': 'choice', 'id': '10', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '10'},
      ]},
      { 'type': 'choice', 'id': '11', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '11'},
      ]},
      { 'type': 'choice', 'id': '12', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '12'},
      ]}, 
      {'type': 'choice', 'id': 'Crystal_system', 'label': 'Choose a crystal system', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '3'},
      ]},
      { 'type': 'choice', 'id': '21', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '3'},
        {'operand': 'Crystal_structure','operator': '==','value': '21'},
      ]},
      { 'type': 'choice', 'id': '22', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '3'},
        {'operand': 'Crystal_structure','operator': '==','value': '22'},
      ]},
      { 'type': 'choice', 'id': '23', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '3'},
        {'operand': 'Crystal_structure','operator': '==','value': '23'},
      ]},
      { 'type': 'choice', 'id': '24', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '3'},
        {'operand': 'Crystal_structure','operator': '==','value': '24'},
      ]},
      { 'type': 'choice', 'id': '25', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '3'},
        {'operand': 'Crystal_structure','operator': '==','value': '25'},
      ]},
      { 'type': 'choice', 'id': '26', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '3'},
        {'operand': 'Crystal_structure','operator': '==','value': '26'},
      ]},
      { 'type': 'choice', 'id': '27', 'label': 'Material', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '3'},
        {'operand': 'Crystal_structure','operator': '==','value': '27'},
      ]}, 
      {'type': 'integer', 'id': 'Lx','label': 'Lx','enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '8'},
        {'operand': '_8','operator': '==','value': '1'},
      ]}, 
      {'type': 'integer', 'id': 'Ly', 'label': 'Ly', 'enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '8'},
        {'operand': '_8','operator': '==','value': '1'},
      ]}, 
      {'type': 'integer','id': 'Number_of_sheet','label': 'Number of sheet','enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '8'},
        {'operand': '_8','operator': '==','value': '1'},
      ]}, 
      {'type': 'string','id': 'Separation_distance','label': 'Layer separation distance (A,bohr,nm)','enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '8'},
        {'operand': '_8','operator': '==','value': '1'},
        {'operand': 'Number_of_sheet', 'operator': '>','value': '1'}
      ]}, 
      {'type': 'boolean', 'id': 'AA_Stacking', 'label': 'AA Stacking','enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '8'},
        {'operand': '_8','operator': '==','value': '1'},
      ]},
      {'type': 'integer','id': 'num_of_unit_cell','label': 'Number of unit celld','enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '8'},
        {'operand': '_8','operator': '==','value': '2'},
      ]},
      {'type': 'integer','id': 'n','label': 'n','enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '8'},
        {'operand': '_8','operator': '==','value': '2'},
      ]}, 
      {'type': 'integer','id': 'm','label': 'm (<=n)','enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '8'},
        {'operand': '_8','operator': '==','value': '2'},
      ]},
      {'type': 'string','id': 'C_C_bong_length','label': 'C-C bond length (A,bohr,nm)','enable': [
        {'operand': 'What_to_do','operator': '==','value': '1'},
        {'operand': 'Crystal_structure','operator': '==','value': '8'},
        {'operand': '_8','operator': 'in','value': '1,2'},
      ]},
      { 'type': 'container',
        'id': '',
        'enable' : [
            {'operand': 'What_to_do','operator': '==','value': '3'},
            {'operand': 'Crystal_system','operator': 'in','value': '21,22,23,24,25,26,27'},
        ],
        'description': 'Enter a number between 0.1nm and 10nm with units of length',
        'label': 'Lattice directions',
        'layout': 'horizontal',
        'children': [ 
          { 'type': 'string', 'id': 'a','label': 'a (A,bohr,nm)', 'enable': [
            {'operand': 'What_to_do','operator': '==','value': '3'},
            {'operand': 'Crystal_system','operator': 'in','value': '21,22,23,24,25,26,27'},
          ]},
          { 'type': 'string', 'id': 'b','label': 'b (A,bohr,nm)', 'enable': [
            {'operand': 'What_to_do','operator': '==','value': '3'},
            {'operand': 'Crystal_system','operator': 'in','value': '21,22,23'},
          ]}, 
          { 'type': 'string', 'id': 'c','label': 'c (A,bohr,nm)', 'enable': [
            {'operand': 'What_to_do','operator': '==','value': '3'},
            {'operand': 'Crystal_system','operator': 'in','value': '21,22,23,24,26'},
          ]}, 
      ]},
      { 'type': 'container',
        'id': '',
        'enable' : [
            {'operand': 'What_to_do','operator': '==','value': '3'},
            {'operand': 'Crystal_system','operator': 'in','value': '21,22,27'},
        ],
        'description': 'Enter a number between -90deg and 90deg with units of angle (deg,grad,rad)',
        'label': 'Lattice angles',
        'layout': 'horizontal',
        'children': [ 
          { 'type': 'string', 'id': 'Alpha', 'label': 'Alpha', 'enable': [
            {'operand': 'What_to_do','operator': '==','value': '3'},
            {'operand': 'Crystal_system','operator': 'in','value': '21,27'},
          ]}, 
          { 'type': 'string', 'id': 'Beta', 'label': 'Beta', 'enable': [
            {'operand': 'What_to_do','operator': '==','value': '3'},
            {'operand': 'Crystal_system','operator': 'in','value': '21,22'},
          ]},
          { 'type': 'string', 'id': 'Gamma', 'label': 'Gamma', 'enable': [
            {'operand': 'What_to_do','operator': '==','value': '3'},
            {'operand': 'Crystal_system','operator': 'in','value': '21'},
          ]},       
      ]},       
    ]
  }
}

#LAYOUT

In [20]:
## Creating the settings panel

In [21]:
url_sim = "https://nanohub.org/api/tools"
#__jupyter_cache = JupyterCache()
Settings = AppBuilder.Settings(
    Project,
    Component, 
    SETTINGS,
    url=url_sim,
    toolname = TOOLNAME,
    layout = LAYOUT['input'],
    #jupyter_cache = __jupyter_cache
)
Settings.content.events["onError"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False},
    { "type": "stateChange", "modifies": STATE_ERROR_OPEN,"newState": True},
    { "type": "stateChange", "modifies": STATE_ERROR_STATUS,"newState": '$e'},
]
Settings.content.events["click"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": True}
]

Settings.content.events["onStatusChange"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_STATUS,"newState": "$e.target.value"}
]
Settings.content.events["onSuccess"]=[
  { "type": "stateChange", "modifies": "open_plot","newState":{'textbook':'primary'} },
  { "type": "propCall2", "calls": "loadVectors","args": ["self", "['^latt_info$']"] },
  { "type": "propCall2", "calls": "loadMolecule","args": ["self", "self.state.moleculeView[0]","self.state.moleculeView[1]"] },
  { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False },  
  { "type": "stateChange", "modifies": "open_results","newState": True},
  { "type": "stateChange", "modifies": "open_params","newState": False},
]


In [22]:
## Creating the output options

In [23]:
RapptureBuilder.loadMolecule(Project, Component)
RapptureBuilder.buildBasis(Project, Component)
RapptureBuilder.buildCrystal(Project, Component)
RapptureBuilder.loadVectors(Project, Component)
Component.addStateVariable("moleculeView", {"type":"array", "defaultValue": [['^structure0$','^structure15$']] })
Component.addPropVariable("onLoadVectors", {"type":"func", 'defaultValue' : '(e,v)=>{e.setState({"ivectors":v.ivectors,"vectors":v.vectors,"atoms":v.atoms, "box":v.box })}'})    
Component.addPropVariable("onLoadBoundary", {"type":"func", 'defaultValue' : '(e,v)=>{e.setState({"boundary":v})}'})    
Component.addPropVariable("onMoleculeView", {"type":"func", 'defaultValue' : '(e,m,p)=>{ e.setState({"moleculeView":[m,p]}); e.props.loadMolecule(e,m,p);}'})    

RESULTS = {
    "textbook" : {
        'title' : 'Textbook basis',
        'action' : { "type": "propCall2", "calls": "onMoleculeView","args": ["self", "['^structure0$','^structure15$']"] },
    },    
    "basis" : {
        'title' : 'Minimal Basis',
        'action' : { "type": "propCall2", "calls": "onMoleculeView","args": ["self", "['^structure2$','^structure10$']","(c,m)=>{return self.props.buildBasis(c,m,self.state.basis_m,self.state.basis_n,self.state.basis_p)}"] },
    },
    "unitcell" : {
        'title' : 'Traditional Basis',
        'action' : { "type": "propCall2", "calls": "onMoleculeView","args": ["self", "['^structure2$','^structure10$']","(c,m)=>{let e=self; return self.props.buildCrystal(c,m,parseFloat(e.state.box[0][0])+parseFloat(e.state.box[0][1])+parseFloat(e.state.box[0][2])-0.005,parseFloat(e.state.box[1][0])+parseFloat(e.state.box[1][1])+parseFloat(e.state.box[1][2])-0.005,parseFloat(e.state.box[2][0])+parseFloat(e.state.box[2][1])+parseFloat(e.state.box[2][2])-0.005)}"] },
    },    
    "crystal" : {
        'title' : 'Crystal',
        'action' : { "type": "propCall2", "calls": "onMoleculeView","args": ["self", "['^structure2$','^structure10$']","(c,m)=>{return self.props.buildCrystal(c,m,self.state.cristal_x,self.state.cristal_y,self.state.cristal_z)}"] },
    },    
}


Results = AppBuilder.Results( 
    Component,
    results = RESULTS,
    #onClick = [{ "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": True }],
    #onLoad = [{ "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False }],
)

basis_callback = {
  "type" : "propCall2", 
  "calls": "loadMolecule",
  "args": [
      "self", 
      "['^structure2$','^structure10$']",
      "(c,m)=>{return self.props.buildBasis(c,m,arguments[1],self.state.basis_n,self.state.basis_p)}"
  ]
}
def basis_callback(**kwargs):
    m = kwargs.get("m","self.state.basis_m")
    n = kwargs.get("n","self.state.basis_n")
    p = kwargs.get("p","self.state.basis_p")
    return {
      "type" : "propCall2", 
      "calls": "loadMolecule",
      "args": [
          "self", 
          "['^structure2$','^structure10$']",
          "(c,m)=>{return self.props.buildBasis(c,m,"+m+","+n+","+p+")}"
      ]
    }

basis_m = MaterialBuilder.GridItem(content=FormHelper.IntSlider( 
  Component,'m', '', 'basis_m', 1, min=1,max=9,step=1,marks=True,
  onChange = basis_callback(m="arguments[1]")
))

basis_n = MaterialBuilder.GridItem(content=FormHelper.IntSlider( 
  Component,'n', '', 'basis_n', 1, min=1,max=9,step=1,marks=True,
  onChange = basis_callback(n="arguments[1]")
))

basis_p = MaterialBuilder.GridItem(content=FormHelper.IntSlider( 
  Component,'p', '', 'basis_p', 1, min=1,max=9,step=1,marks=True,
  onChange =  basis_callback(p="arguments[1]")
))

def cristal_callback(**kwargs) :
    x = kwargs.get("x","self.state.cristal_x")
    y = kwargs.get("y","self.state.cristal_y")
    z = kwargs.get("z","self.state.cristal_z")
    return {
      "type" : "propCall2", 
      "calls": "loadMolecule",
      "args": [
          "self", 
          "['^structure2$','^structure10$']",
          "(c,m)=>{return self.props.buildCrystal(c,m,"+x+","+y+","+z+")}"
      ]
    }
cristal_x = MaterialBuilder.GridItem(content=FormHelper.NumberSlider( 
  Component,'Dimension X', '', 'cristal_x', 1.5, min=0.1,max=2.5,step=0.1,marks=True,
  onChange =  cristal_callback(x="arguments[1]")
))

cristal_y = MaterialBuilder.GridItem(content=FormHelper.NumberSlider( 
  Component,'Dimension Y', '', 'cristal_y', 1.5, min=0.1,max=2.5,step=0.1,marks=True,
  onChange =  cristal_callback(y="arguments[1]")
))

cristal_z = MaterialBuilder.GridItem(content=FormHelper.NumberSlider( 
  Component,'Dimension Z', '', 'cristal_z', 1.5, min=0.1,max=2.5,step=0.1,marks=True,
  onChange =  cristal_callback(z="arguments[1]")
))
Results.addContent(
    FormHelper.ConditionalGroup(
        Component, [basis_m, basis_n, basis_p], "open_plot", [{'operation' : "==", "operand" : 'basis'}]
    )   
)
Results.addContent(
    FormHelper.ConditionalGroup(
        Component, [cristal_x, cristal_y, cristal_z], "open_plot", [{'operation' : "==", "operand" : 'crystal'}]
    )   
)

In [24]:
## Create Planes Sliders

In [25]:
Colors = AppBuilder.ColorSliders(
    Project,
    Component, 
    sliders = {
        "100" : {
          'color':'#a6cee3',
          'label':'Plane YZ',
          'plane':[1,0,0],
        },
        "010" : {
          'color':'#a6cee3',
          'label':'Plane XZ',
          'plane':[0,1,0],
        },
        "001" : {
          'color':'#a6cee3',
          'label':'Plane XY',
          'plane':[0,0,1],
        },   
        "miller" : {
          'color':'#a6cee3',
          'label':'Miller Plane',
          'plane':[1,0,0],
          'options':{
             "100":[1,0,0],
             "111":[1,1,1],
             "121":[1,2,1],
             "010":[0,1,0],
             "001":[0,0,1],
             "011":[0,1,1],
             "110":[1,1,0],
          },
        },
    }
)

In [26]:
## Creating Colorschema and adding nanoHUB logo

In [27]:
Theme = MaterialBuilder.DefaultTheme(
    primary_color = '#699FBB',
    secondary_color = '#f1f1f1',
    primary_bg = '#FFFFFF',
    secondary_bg = '#dbeaf0',
    default_button = 'rgba(255, 255, 255, 0.87)',
    primary_button = 'rgba(255, 255, 255, 0.87)',
    secondary_button = 'rgba(0, 0, 0, 0.87)',
    default_button_bg = 'rgb(63, 162, 192)',
    primary_button_bg = 'rgba(0, 0, 0, 0.65)',
    secondary_button_bg = 'rgba(0, 0, 0, 0.12)',
    drawer_position = "relative"
)
Theme['overrides']['MuiDrawer']['paper']['margin-top'] = '0px'
Theme['overrides']['MuiDrawer']['paper']['height'] = '100%'
ThemeProvider = MaterialBuilder.ThemeProvider( Component, Theme )

AppBar = MaterialBuilder.AppBar(
    state="DrawerIsVisible", 
    #styles=("AppBarStyle", ["bar_shifted", "bar_normal"]),
    title="Crystal Viewer"
)

logo = TeleportElement(TeleportContent(elementType="img"))
logo.content.attrs["width"] = "120"
logo.content.attrs["src"] = "https://nanohub.org/app/site/media/images/PressKit/nanoHUB_logo_color.jpg"
AppBar.content.children[0].addContent(logo)

In [28]:
## Creating Expansion panels

In [29]:
#Project.globals.assets.append({"type": "style", "content": ".bar_shifted {margin-left:350px}"})
#Project.globals.assets.append({ "type": "style", "content": ".bar_normal {margin-left:0px}"})
ExpansionPanel = MaterialBuilder.ExpansionPanel(
    title="Results", 
    content=[Results],
    expanded={
      "type": "dynamic",
      "content": {
        "referenceType": "state",
        "id": "open_results"
      }    
    },     
)
ExpansionPanel.content.events['change'] = [{ "type": "stateChange", "modifies": "open_results","newState": "$toggle"}]
Drawer.addContent(ExpansionPanel)

PlanesPanel = MaterialBuilder.ExpansionPanel(
    title="Planes", 
    expanded={
      "type": "dynamic",
      "content": {
        "referenceType": "state",
        "id": "open_planes"
      }    
    }, 
    content=[Colors]
)
PlanesPanel.content.events['change'] = [{ "type": "stateChange", "modifies": "open_planes","newState": "$toggle"}]
Drawer.addContent(PlanesPanel)


ParametersPanel = MaterialBuilder.ExpansionPanel(
    title="Parameters", 
    expanded={
      "type": "dynamic",
      "content": {
        "referenceType": "state",
        "id": "open_params"
      }    
    }, 
    content=[Settings]
)
ParametersPanel.content.events['change'] = [{ "type": "stateChange", "modifies": "open_params","newState": "$toggle"}]
Drawer.addContent(ParametersPanel)


In [30]:
## Creating reset button

In [31]:
Gridt = TeleportElement(MaterialContent(elementType="Grid"))
Gridt.content.attrs["color"] = "secondary"
Gridt.content.attrs["container"] = True
Gridt.content.attrs["direction"] = "column"
resetSettings = {}
for k,v in SETTINGS.items():
    resetSettings[k] = v["default_value"]

Buttontt= MaterialBuilder.Button(
      title = "Reset Setting", 
      variant = "text", 
      style = {'backgroundColor':'#999999', 'borderRadius' : '0px', 'minHeight':'40px', 'color':'rgba(255, 255, 255, 0.87)'}, 
      onClickButton=[{ "type": "stateChange", "modifies": "parameters","newState": resetSettings}]
)
Gridt.addContent(Buttontt)
Drawer.addContent(Gridt)

In [32]:
## Creating ploting component

In [33]:
BasePlot = PlotlyBuilder.Molecule( Project, Component, style_state="AppBarStyle")

In [34]:
ThemeProvider.addContent(Login)

In [35]:
## Assembling the app

In [36]:
Gridh = TeleportElement(MaterialContent(elementType="Grid"))
Gridh.content.attrs["container"] = True
Gridh.content.attrs["direction"] = "row" 
Gridh.addContent(Drawer)
Gridh.addContent(BasePlot)  

Gridv = TeleportElement(MaterialContent(elementType="Grid"))
Gridv.content.attrs["container"] = True
Gridv.content.attrs["direction"] = "column"
Gridv.addContent(AppBar)
Gridv.addContent(Gridh)

ThemeProvider.addContent(Gridv)
ThemeProvider.addContent(Loader)
ThemeProvider.addContent(ErrorMessage)

Component.addNode(ThemeProvider)


In [37]:
## Displaying the app in IFRAME

In [38]:
from nanohub.uidl.ipywidgets import buildWidget
BasePlot.content.style = {
  "position": "absolute",
  "height": "700px", #calc(100vh - 64px)
  "width": "100%", #calc(100vh - 64px)
}
Drawer.content.style = {
  "height": "700px",
}

#MyComponentWidget = buildWidget(Project, jupyter_axios=True, debugger=False, verbose=False);


In [39]:
a = MyComponentWidget()
a

HTML(value="<link rel='stylesheet' href='https://fonts.googleapis.com/icon?family=Material+Icons'/><style>.p-W…

<IPython.core.display.Javascript object>

CrystalViewerAppWidget(boundary=[[0, 0, 0], [1, 1, 1]], config={'displayModeBar': True, 'responsive': True}, m…

In [40]:
#Project.buildReact( TOOLNAME+"PROD"+".html");
#from IPython.core.display import display
#from IPython.display import IFrame
#display(IFrame(src=TOOLNAME+"PROD"+".html", width="100%", height="800"))